# Introduction to Data Science final project

By: Yehhuda Benais and Tal Tubul

# **imports**

In [42]:
#imports:
import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
import re
import os.path
import csv
import ast
import math
import statistics as stat

# **crawling functions**

##### The next function handeling the data we get from the functions: 
```python
get_tma() 
```
```python
get_Construction_evacuation_complexes()
```
The function gets a list of addresses and returns:

   **case 1:** street's name 23-25 ==> [street's name 23, street's name 24, street's name 25]

   **case 2:** street's name 23,24 ==> [street's name 23, street's name 24]

   **case 3:** street's name 23 ==> [street's name 23]
   
   **case 4:** street's name ==> [street's name]

The function have been writed with chatGPT.

In [29]:
def fix_addresses(addresses):
    fixed_addresses = []
    for address in addresses:
        # remove number between parentheses
        address = re.sub(r'\(\d+\)', '', address)
        # split address by hyphen
        hyphen_split = address.split('-')
        for i in range(0, len(hyphen_split)):
            hyphen_split[i] = hyphen_split[i].strip()
        if len(hyphen_split) == 2:
            # check if both sides of hyphen are numbers
            if hyphen_split[0][-1].isdigit() and hyphen_split[1].isdigit() or (not hyphen_split[1].isalpha() and not any(c.isalpha() for c in hyphen_split[1])):
                start_num = int(re.sub(r'\D', '', hyphen_split[0]))
                end_num = int(re.sub(r'\D', '', hyphen_split[1]))
                # add new addresses to fixed_addresses
                for num in range(start_num, end_num + 1):
                    fixed_addresses.append(hyphen_split[0].strip('1234567890,-()') + ' ' + str(num))
            else:
                fixed_addresses.append(address)
        else:
            # check if address has numbers separated by comma
            comma_split = address.split(',')
            if len(comma_split) > 1:
                addresses = re.split('\s*,\s*', address)
                fixed_addresses.append(addresses[0])
                home = ''.join([char for char in address if char.isalpha() or char.isspace()])
                #home = address.strip('1234567890,-()')
                addresses.pop(0)
                for addr in addresses:   
                    fixed_addresses.append(f'{home} {addr}')
            else: 
                fixed_addresses.append(address)    
    return fixed_addresses

### **tma 38 projects crawlings:**
##### the next two functions get the URL of a city 
##### and return a list with all the adresses that have tma 38 project from the current URL's page


In [6]:
def get_tma(url):
# Send a GET request to the URL
    response = requests.get(url)
    # Parse the HTML content of the response using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    # zFind the table with the specified ID
    table = soup.find("table", {"class": "gridView"})
    # Find all the rows in the table and extract the values in the "כתובת" column
    rows = table.find_all("tr")
    addresses_p = []
    for row in rows:
        cells = row.find_all("td", attrs={"class","gv_cc"})
        if len(cells) >= 3:
            addresses_p.append(cells[1].text)
        cells = row.find_all("td", attrs={"class","gv_cc2"})
        if len(cells) >= 3:
            addresses_p.append(cells[1].text)
    addresses_p = [s.strip('\r\n\t') for s in addresses_p]
    return fix_addresses(addresses_p)

##### The next function gets city name and crawling from the website: tma-38, 
 all the tma-38 projects that are maded and projecets that are now at proccesses at the city. 
 **The function uses the next URL's:**
 ##### The next URL is to get all the tma 38 progects from type-1 that are currently in proccesses:
 ["https://www.t-m-a38.co.il/Default.aspx?gvprojects__gvac=1&gvprojects__gvff0={city}&gvprojects__gvfl0=0&gvprojects__gvpi={i}&tabid=65"](https://)
 ##### The next URL is to get all the tma 38 progects from type-2 that are currently in proccesses:
 ["https://www.t-m-a38.co.il/Default.aspx?gvprojects__gvac=1&gvprojects__gvff0={city}&gvprojects__gvfl0=0&gvprojects__gvpi={i}&tabid=118"](https://)
 ##### The next URL is to get all the tma 38 progects that have been ended thier proccesses:
 ["https://www.t-m-a38.co.il/Default.aspx?gvprojects__gvac=1&gvprojects__gvff0={city}&gvprojects__gvfl0=0&gvprojects__gvpi={i}&tabid=96"](https://)

 The function return a dict with all the adresses that have tma 38 project with the next values:

 1 ==> if the progect is during tma38 process

 2 ==> if the progect is during tma38 type 2 process 
 
 3 ==> if the progect is ended the tma38 process

 if the customer entered a wrong value of city the functions will return an empty dict.

In [7]:
def tma_df(city):
    unfinished_tma = []
    unfinished_tma_type2 = []
    finished_tma = []
    for i in range(15):
        url = f"https://www.t-m-a38.co.il/Default.aspx?gvprojects__gvac=1&gvprojects__gvff0={city}&gvprojects__gvfl0=0&gvprojects__gvpi={i}&tabid=65"
        unfinished_tma += get_tma(url)
        url = f"https://www.t-m-a38.co.il/Default.aspx?gvprojects__gvac=1&gvprojects__gvff0={city}&gvprojects__gvfl0=0&gvprojects__gvpi={i}&tabid=118"
        unfinished_tma_type2 += get_tma(url)
        url = f"https://www.t-m-a38.co.il/Default.aspx?gvprojects__gvac=1&gvprojects__gvff0={city}&gvprojects__gvfl0=0&gvprojects__gvpi={i}&tabid=96"
        finished_tma += get_tma(url)
    string_dict = {s: i for i, strings in enumerate([unfinished_tma, unfinished_tma_type2, finished_tma], start=1) for s in strings}
    # Print dictionary
    return string_dict

### **Construction evacuation projects crawlings:**
##### The next function get's city's name and the URL of the goverment's website for Construction evacuation projects
##### and return a list with all the adresses that have been part of construction evacuation project at the current city.

In [30]:
def get_Construction_evacuation_complexes(city_name):
    # Set up the webdriver
    driver = webdriver.Chrome()
    # Navigate to the job search website
    driver.get("https://www.gov.il/apps/moch/viewlist/renew/mitchamim-rashut")
    # Wait for the paginator to load
    paginator = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "p-paginator")))
    #Find the search/filter input element and enter the city name
    time.sleep(1.2)
    search_input = driver.find_element(By.CSS_SELECTOR, "#search")
    search_input.send_keys(city_name)
    time.sleep(1)
    data = []
    # Click all "Next" buttons until there is no button left
    while True:
        next_button = paginator.find_element(By.CSS_SELECTOR, ".ui-paginator-next")
        tbody = None
        tbody = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME,"tbody")))
        try:
            time.sleep(1)
            temp = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,"column-value")))
        except:
           print("invalid input try again...") 
           return []
        if tbody:
            some = tbody.find_elements(By.CSS_SELECTOR, "td")
            for value in some:
                data.append(value.text)
        if "ui-state-disabled" in next_button.get_attribute("class"):
            # No more next buttons, exit loop
            break
        next_button.click()  
    # Close the webdriver
    driver.quit()
    for d in range(0, len(data)):
        if ('יישוב: ' + city_name) in data[d]:
           fixed_data = data[(d+1)::9]
           break
    return  fix_addresses(fixed_data)

### **Tabu crawling:**
#### The function get the full address of an apartment and return if she have a tabu or not.
#### The function uses the next URL for crawling: https://al-harishum.moch.gov.il/
##### Due to a blocking system at the current website we coulded crawl the website...

In [ ]:
def get_tabu(city, street, number):
    strongs = None
    driver = webdriver.Chrome()
    driver.get("https://al-harishum.moch.gov.il/")
    
    # Wait for the inputs to load:
    _city = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_SearchPanel_TabContainer1_TabPanelAddress_txtYeshuv")))
    _number = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_SearchPanel_TabContainer1_TabPanelAddress_txtBait")))
    _street = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_SearchPanel_TabContainer1_TabPanelAddress_txtRechov")))
    
    # Enter values into the inputs:
    _city.send_keys(city)
    _number.send_keys(number)
    _street.send_keys(street)
    
    # Click the search button:
    button = driver.find_element(By.ID, "ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_SearchPanel_TabContainer1_TabPanelAddress_btnAddress")
    button.click()
    
    # Check if the invalid state element exists:
    try:
        strongs = driver.find_elements(By.TAG_NAME, "strong")
        strong = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_NoBotInvalidState")))
        
    except:
        pass
    
    driver.quit()
    return strongs
res = get_tabu('אשדוד', 'הר כנען 2', '2')
for r in res:
    print(r.text)

### **Yad-2 crawling:**
##### The function get's the room's number and a URL for a city and return a meta data with all the apartments for sale at the current city with the next properties: 
prices, streets, cities, floors, sizes, terraces, floors_in_building, parkings, aircondition, bars, elevators, water_headers, accessibility, shelters, renovated, storages, tadiran, furnitures, status, time_enter, sea_view, sukka.
##### The function uses the next URLs:
**Jerusalem**:https://www.yad2.co.il/realestate/forsale?topArea=100&area=7&city=3000&rooms=3-3

**Tel-aviv**: https://www.yad2.co.il/realestate/forsale?topArea=2&area=1&city=5000&rooms=3-3

**Haifa**: https://www.yad2.co.il/realestate/forsale?topArea=25&area=5&city=4000&rooms=3-3

**Rishon-lezion**: https://www.yad2.co.il/realestate/forsale?topArea=2&area=9&city=8300&rooms=3-3

**Petah-tikva**: https://www.yad2.co.il/realestate/forsale?topArea=2&area=4&city=7900&rooms=3-3

**Netanya**: https://www.yad2.co.il/realestate/forsale?topArea=19&area=17&city=7400&rooms=3-3

**Holon**: https://www.yad2.co.il/realestate/forsale?topArea=2&area=11&city=6600&rooms=3-3

**Beer-sheba**: https://www.yad2.co.il/realestate/forsale?topArea=43&area=22&city=9000&rooms=3-3

**Bnei-brakh**: https://www.yad2.co.il/realestate/forsale?topArea=2&area=78&city=6100&rooms=3-3

**Ashdod**: https://www.yad2.co.il/realestate/forsale?topArea=41&area=21&city=0070&rooms=3-3

In [9]:
def get_info(rooms):
    prices = [] 
    streets = []
    cities = []
    floors = [] 
    sizes = []
    terraces = [] 
    floors_in_building = [] 
    parkings = []
    aircondition = []
    bars = [] 
    elevators = []
    water_headers = [] 
    accessibility = [] 
    shelters = [] 
    renovated = []
    storages = []
    tadiran = []
    flexiable = []
    furnitures = [] 
    status = []
    time_enter = []
    sea_view = []
    sukka = []
    to_text = []
    city = []
    driver = webdriver.Edge()
    driver.get(f"https://www.yad2.co.il/realestate/forsale?topArea=100&area=7&city=3000&property=1&rooms={rooms}-{rooms}")
    pages = WebDriverWait(driver, 80).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".page-num")))
    if pages:
        for page in range(1,int(pages.pop().text)+1):
            driver.get(f"https://www.yad2.co.il/realestate/forsale?topArea=100&area=7&city=3000&property=1&rooms={rooms}-{rooms}&page={page}")
            pages = WebDriverWait(driver, 80).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".page-num"))) 
            index = 0
            while index < 39:
                try:
                    ad_button = driver.find_element(By.ID, 'slot-close-button')
                    ad_button.click()
                except:
                    pass
                item = driver.find_element(By.ID, f"feed_item_{index}")
                try:
                    time.sleep(0.8)
                    item.click()
                    waitt = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.ID,f'accordion_wide_{index}')))
                    time.sleep(1.2)
                    try:
                        city = item.find_element(By.CLASS_NAME, "subtitle").text
                        if city:
                            cities.append(city)
                        else: cities.append('nan')
                    except:
                        cities.append('nan')
                    street = item.find_element(By.ID, f"feed_item_{index}_title").text
                    if street:
                        streets.append(street)
                    else: streets.append('nan')
                    floor = driver.find_element(By.ID, f"data_floor_{index}").text
                    if floor:
                        floors.append(floor)
                    else: floors.append('nan')
                    size = driver.find_element(By.ID, f"data_SquareMeter_{index}").text
                    if size:
                        sizes.append(size)
                    else: sizes.append('nan')
                    price = driver.find_element(By.ID, f"feed_item_{index}_price").text
                    if price:
                        prices.append(price)
                    else: prices.append('nan')
                    waitt = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.ID,f'accordion_wide_{index}')))
                    data_list = item.find_elements(By.TAG_NAME,'dl')
                    if data_list:
                        flag = 0
                        for d in data_list:
                            to_text.append(d.text)
                        for t in to_text:
                            if 'מצב הנכס' in t:
                                status.append(t)
                                flag = 1
                                break 
                        if flag == 0:
                            status.append('none')
                        else: flag = 0
                        for t in to_text:
                            if 'תאריך כניסה' in t:
                                time_enter.append(t)
                                flag = 1
                                break 
                        if flag == 0:
                            time_enter.append('none')
                        else: flag = 0        
                        for t in to_text:
                            if 'מרפסות' in t:
                                terraces.append(t)
                                flag = 1
                                break 
                        if flag == 0:
                            terraces.append('none')
                        else: flag = 0      
                        for t in to_text:
                            if 'חניות' in t:
                                parkings.append(t)
                                flag = 1
                                break 
                        if flag == 0:
                            parkings.append('none')
                        else: flag = 0
                        for t in to_text:
                            if 'קומות בבנין' in t:
                                floors_in_building.append(t)
                                flag = 1
                                break 
                        if flag == 0:
                            floors_in_building.append('none')
                        else: flag = 0
                        to_text.clear() 
                    else:
                        status.append('nan')
                        parkings.append('nan')
                        floors_in_building.append('nan')
                        terraces.append('nan')
                        time_enter.append('nan')
                    contect = item.find_elements(By.CLASS_NAME,"show-more-container")
                    if contect:
                        if ('הים' in contect[0].text or 'לים' in contect[0].text) and 'נוף' in contect[0].text:
                            sea_view.append(1)
                        else: sea_view.append(0)
                        if 'סוכה' in contect[0].text and 'מרפסת' in contect[0].text:
                            sukka.append(1)
                        else: sukka.append(0)
                    else: 
                        sea_view.append(0)
                        sukka.append(0)
                    elements2 = item.find_elements(By.CSS_SELECTOR, 'div.items_container > div.info_feature')
                    time.sleep(0.5)
                    if elements2:
                        for element in elements2:
                            class_name = element.get_attribute('class')
                            if class_name == 'info_feature':
                                result = 1
                            elif class_name == 'info_feature delete':
                                result = 0
                            text = element.text
                            if 'נכס בבלעדיות' in text or 'מטבח כשר' in text:
                                continue
                            if 'מיזוג' in text:
                                aircondition.append(result)
                                continue
                            elif 'סורגים' in text:
                                bars.append(result)
                                continue
                            elif 'מעלית' in text:
                                elevators.append(result)
                                continue
                            elif 'דוד שמש' in text:
                                water_headers.append(result)
                                continue
                            elif 'גישה לנכים' in text:
                                accessibility.append(result)
                                continue
                            elif 'ממ"ד' in text:
                                shelters.append(result)
                                continue
                            elif 'משופצת' in text:
                                renovated.append(result)
                                continue
                            elif 'מחסן' in text:
                                storages.append(result)
                                continue
                            elif 'מזגן תדיראן' in text:
                                tadiran.append(result)
                                continue
                            elif 'ריהוט' in text:
                                furnitures.append(result)
                                continue
                            elif 'גמיש' in text:
                                flexiable.append(result)
                                continue
                    else:
                        flexiable.append('nan')  
                        furnitures.append('nan') 
                        tadiran.append('nan') 
                        renovated.append('nan') 
                        shelters.append('nan') 
                        accessibility.append('nan') 
                        water_headers.append('nan') 
                        elevators.append('nan') 
                        bars.append('nan') 
                        aircondition.append('nan')
                        storages.append('nan') 
                    index += 1
                except:
                    pass
            print(f'{len(streets)} {len(prices)} {len(cities)} {len(sizes)} {len(floors)} {len(furnitures)}  {len(tadiran)} {len(bars)} {len(floors_in_building)} {len(parkings)} {len(sea_view)}')
            data = {
                'prices': prices,
                'streets': streets,
                'cities': cities,
                'floors': floors,
                'sizes': sizes,
                'terraces': terraces,
                'floors_in_building': floors_in_building,
                'parkings': parkings,
                'aircondition': aircondition,
                'bars': bars,
                'elevators': elevators,
                'water_headers': water_headers,
                'accessibility': accessibility,
                'shelters': shelters,
                'renovated': renovated,
                'storages': storages,
                'tadiran': tadiran,
                'furnitures': furnitures,
                'status': status,
                'time_enter': time_enter,
                'sea_view': sea_view,
                'sukka': sukka
            }
            df = pd.DataFrame(data)
            filename = 'C:\\Users\\Tal\\Desktop\\לימודים\\מדעי נתונים\\jerusalem_df_file_3_rooms.csv'
            #write data to the file:
            file_exists = os.path.isfile(filename)
            with open(filename, 'a', newline='') as csvfile:
                fieldnames = ['prices', 'streets', 'cities', 'floors', 'sizes', 'terraces', 'floors_in_building', 'parkings', 'aircondition', 'bars', 'elevators', 'water_headers', 'accessibility', 'shelters', 'renovated', 'storages', 'tadiran', 'furnitures', 'status', 'time_enter', 'sea_view', 'sukka']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                if not file_exists:
                    writer.writeheader()
                writer.writerow(data)
            for l in [prices, streets, cities, floors, sizes, terraces, floors_in_building, parkings, aircondition, bars, elevators, water_headers, accessibility, shelters, renovated, storages, tadiran, furnitures, status, time_enter, sea_view, sukka]:
                l.clear()

##### The next function gets a file name of meta data with all the apartments of a city for sale in Yad-2 and creates a dataframe from it:
###### The function were written with chatGPT

In [10]:
def reorder_df(file_name):
    df = pd.read_csv(file_name, encoding='hebrew')
    #finds the number of apartments at the meta data df:
    x = 0
    for row in range(0, df.shape[0]):
        x += len(df.loc[row, 'sea_view'].split(','))
    indexes = list(range(1, x+1))
    new_df = pd.DataFrame({'index':indexes}) 
    #creates a regular df:
    for column in df.columns:
        l = []
        for row in range(0, df.shape[0]):
            l += ast.literal_eval(df.loc[row, column].strip('\\n'))
        l = [i for i in l if i != '']  
        new_df[column] = l
    return new_df

##### The next function calls to reorder_df() and creates files of dataframes for each city at the cities list:

In [ ]:
def fix_df():
    cities = ['jerusalem', 'tel-aviv', 'haifa', 'rishon-lezion', 'petah-tikva', 'netanya', 'holon', 'beer-sheva', 'ashdod','bnei-brakh']
    for i in range(0, len(cities)):
        src = f'C:\\Users\\Tal\\Desktop\\לימודים\\מדעי נתונים\\{cities[i]}_df_file_3_rooms.csv'
        new_df = reorder_df(src)
        new_df.to_csv(f'C:\\Users\\Tal\\Desktop\\לימודים\\מדעי נתונים\\apartments_info\\{cities[i]}_df_file_3_rooms.csv', encoding='hebrew')
        print(cities[i])
fix_df()    

### **ad-nadlan crawling:**
##### The next function gets URL and crawl all the data about all the deals that have been maked in that city from 2010 to 2023:

In [8]:
def get_deals(url):
    data = {'city':[],'street':[],'rooms':[],'size':[],'floor':[],'date_builded':[],
            '2010':[],'2011':[],'2012':[],'2013':[],'2014':[],'2015':[],'2016':[],
            '2017':[], '2018':[], '2019':[], '2020':[], '2021':[], '2022':[],'2023':[]
    }
    df = pd.DataFrame(data)        
    user_agent= {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)
    soup = BeautifulSoup(response.content, "html.parser")
    streets = []
    flag = 1
    while(flag):
        vals = soup.find_all("td")
        t_df = pd.DataFrame(data)
        row_index = 1
        for i, street in enumerate(vals[2::9]):
            date = vals[(i * 9 + 0)].text
            year = date[-4:]
            if street.text == '':
                continue
            else:
                streets.append(street.text)
                row =pd.DataFrame({
                    'city':vals[i * 9 + 1].text,'street': vals[i * 9 + 2].text,
                    'rooms':vals[i * 9 + 3].text,'size':vals[i * 9 + 4].text,
                    'floor':vals[i * 9 + 5].text,'date_builded':vals[i * 9 + 8].text,
                    '2010':'none','2011':'none','2012':'none','2013':'none','2014':'none',
                    '2015':'none','2016':'none','2017':'none', '2018':'none', '2019':'none', 
                    '2020':'none','2021':'none', '2022':'none','2023':'none'}, index=[row_index])    
                row[year] = vals[i * 9 + 6].text #to get the full price enter : vals[i * 9 + 6].text
                t_df = pd.concat([row,t_df.loc[:]], ignore_index=True)
                row_index += 1
        df = pd.concat([df, t_df])
        df = df.reset_index(drop=True)
        link = soup.find_all('a',attrs={'class','page-link text-nowrap px-3 py-2 rounded-pill'}).pop()
        if 'הבא' in link.text:
            next_page = 'https://www.ad.co.il' + soup.find_all('a',attrs={'class','page-link text-nowrap px-3 py-2 rounded-pill'}).pop()['href']
            response = requests.get(next_page,headers=user_agent)
            soup = BeautifulSoup(response.content, "html.parser")
        else: flag = 0    
    return df

##### The next function call to get_deals() function to crawl information about all the deals that have been maked in each city from the cities list and creates a data frame for each city.
##### links for last deals in real estate:
**Jerusalem**: https://www.ad.co.il/nadlanprice?rooms=3,3&city=jerusalem&year=2010,2023&area=jerusalem-maale-adummim&pageindex=1

**Tel-aviv**: https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&area=tel-aviv-area&city=tel-aviv&pageindex=1

**Haifa**: https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&area=haifa-hof-hacarmel&city=haifa&pageindex=1

**Rishon-lezion**: https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&area=rishon-lezion-area&city=rishon-lezion&pageindex=1

**Petah-tikva**: https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&area=petah-tikva-area&city=petah-tikva,givat-hashlosha&pageindex=1

**Netanya**: https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&city=netanya&area=netanya-area&pageindex=1

**Holon**:https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&city=netanya&area=netanya-area&pageindex=1

**Beer-sheba**:  https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&city=beersheba&area=beer-sheva-area&pageindex=1

**Bnei-brakh**:https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&city=bnei-brakh&area=ramat-gan-givatayim&pageindex=1

**Ashdod**:https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&area=ashdod-ashkelon&city=ashdod&pageindex=1



In [ ]:
def create_deals_files():
    urls = ['https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&city=bnei-brakh&area=ramat-gan-givatayim&orderby=4'
            'https://www.ad.co.il/nadlanprice?rooms=3,3&city=jerusalem&year=2010,2023&area=jerusalem-maale-adummim&orderby=4',
            'https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&area=tel-aviv-area&city=tel-aviv&orderby=4',
            'https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&area=haifa-hof-hacarmel&city=haifa&orderby=4',
            'https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&area=rishon-lezion-area&city=rishon-lezion&orderby=4',
            'https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&area=petah-tikva-area&city=petah-tikva,givat-hashlosha&orderby=4',
            'https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&city=netanya&area=netanya-area&orderby=4',
            'https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&city=holon&area=holon-area&orderby=4',
            'https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&city=beer-sheva&area=beer-sheva-area&orderby=4',
            'https://www.ad.co.il/nadlanprice?rooms=3,3&year=2010,2023&area=ashdod-ashkelon&city=ashdod&orderby=4',
        ]    
    cities = ['jerusalem', 'tel-aviv', 'haifa', 'rishon-lezion', 'petah-tikva', 'netanya', 'holon', 'beer-sheva', 'ashdod','bnei-brakh']
    for i, url in enumerate(urls):
        result = get_deals(url)
        result.drop_duplicates()
        print(cities[i])
        result.to_csv(f'C:\\Users\\Tal\\Desktop\\לימודים\\מדעי נתונים\\last_deals_expirience\\{cities[i]}_3_rooms.csv', encoding='hebrew')
create_deals_files()

##### The next function summerize the data frames of the last deals by calculating the average sum for all the deals per a year for each building:

In [ ]:

def clean_deals_df():
    cities = ['jerusalem', 'tel-aviv', 'haifa', 'rishon-lezion', 'petah-tikva', 'netanya', 'holon', 'beer-sheva', 'ashdod','bnei-brakh']
    data = {'city':[],'street':[],'rooms':[],'size':[],'floor':[],'date_builded':[],
            '2010':[],'2011':[],'2012':[],'2013':[],'2014':[],'2015':[],'2016':[],
            '2017':[], '2018':[], '2019':[], '2020':[], '2021':[], '2022':[],'2023':[]
    }
    for city in cities:
        src = f'C:\\Users\\Tal\\Desktop\\לימודים\\מדעי נתונים\\last_deals_expirience\\{city}_3_rooms.csv'
        df = pd.read_csv(src, encoding='hebrew')
        df.drop_duplicates(subset=['city', 'street', 'rooms', 'size', 'floor', 'date_builded', '2010', '2011', '2012', '2013', '2014', '2015',
                                    '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'], inplace=True)
                                    
        df = df.reset_index(drop=True)
        new_df = pd.DataFrame(data)
        count= [1 for _ in range(14)] 
        r = 0
        while r < (df.shape[0]-1):
            index = 1
            row = [df.loc[r, 'city'], df.loc[r, 'street'], df.loc[r, 'rooms'], df.loc[r, 'size'], df.loc[r, 'floor'], df.loc[r, 'date_builded'],
                   'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none']
            while(0 < index < (df.shape[0])):
                if (r + index < df.shape[0]) and df.loc[r + index, 'street'] in df.loc[r, 'street']  : 
                        for year in range(2010, 2024):
                            s1 = str(df.loc[r, f'{year}'])
                            s2 = str(df.loc[r + index, f'{year}'])
                            if s1 != 'none' and s2 != 'none' :
                                if math.isnan(float(s2.replace("," , ""))) :
                                    df.loc[r + index, f'{year}'] = 'none'
                                    break
                                elif math.isnan(float(s1.replace("," , ""))) :
                                    df.loc[r + index, f'{year}'] = 'none'
                                    break
                                else:
                                    df.loc[r, f'{year}'] = int(s1.replace("," , "")) + int(s2.replace("," , ""))
                                    count[year-2010] += 1
                                    break
                            elif df.loc[r + index, f'{year}'] != 'none' and  (df.loc[r, f'{year}'] == 'none' or math.isnan(float(s1.replace("," , ""))) == True):
                                row[year-2004] =  df.loc[r + index, f'{year}']
                                df.loc[r, f'{year}'] = df.loc[r + index, f'{year}']
                                break
                        index += 1
                else:
                    for i, c in enumerate(count):
                        if c > 1:
                            row[i + 6] = int(df.loc[r, f'{i+2010}'] / c)                  
                    count= [1 for _ in range(14)]
                    new_df.loc[len(new_df.index)] = row
                    r += index
                    index = 0
        print(city)
        new_df.to_csv(f'C:\\Users\\Tal\\Desktop\\לימודים\\מדעי נתונים\\last_deals_expirience\\fixed_{city}_3_rooms.csv', encoding="hebrew")
clean_deals_df()

## **creating final files**
#### The next functions taking care the for all the data frames that we crawled and creating files from them of a one data frame per each city from the cities list.

##### The following function is designed to handle the addresses that appear in each data frame we built so that we can merge all the data frames together,the function remove the chars: /.'- from the text.

In [38]:
def remove_characters(strings):
    if type(strings) == list:
        result = []
        for string in strings:
            cleaned_string = string.replace('-', ' ').replace("'", '').replace('/', '').replace('"', '').replace('.', '').replace('יי', 'י').replace('וו', 'ו')
            cleaned_string = cleaned_string.split(" ")
            if 'מפה' in cleaned_string:
                cleaned_string.remove('מפה')
            if 'שד' in cleaned_string:
                cleaned_string.remove('שד')
            if 'שדרות' in cleaned_string:
                cleaned_string.remove('שדרות')
            cleaned_string.sort(reverse=True)
            cleaned_string = [k.strip() for k in cleaned_string]
            string_template = ' '.join(cleaned_string)
            result.append(string_template.strip())
            result.sort()
    elif type(strings) == dict:
        result = {}
        for key, value in strings.items():
            cleaned_string = key.replace('-', ' ').replace("'", '').replace('/', ' ').replace('"', '').replace('.', ' ').replace('יי', 'י').replace('וו', 'ו')
            cleaned_string = cleaned_string.split(" ")
            if 'מפה' in cleaned_string:
                cleaned_string.remove('מפה')
            if 'שד' in cleaned_string:
                cleaned_string.remove('שד')
            if 'שדרות' in cleaned_string:
                cleaned_string.remove('שדרות')
            cleaned_string.sort(reverse=True)
            cleaned_string = [k.strip() for k in cleaned_string]
            string_template = ' '.join(cleaned_string)
            result[string_template.strip()] = value
        result = dict(sorted(result.items()))
    elif type(strings) == type(pd.DataFrame()):
        result = strings
        for index, street in enumerate(result['streets']):
            cleaned_string = street.replace('-', ' ').replace("'", '').replace('/', ' ').replace('"', '').replace('.', ' ').replace('יי', 'י').replace('וו', 'ו')
            cleaned_string = cleaned_string.split(" ")
            cleaned_string = [k.strip() for k in cleaned_string]
            if 'מפה' in cleaned_string:
                cleaned_string.remove('מפה')
            if 'שד' in cleaned_string:
                cleaned_string.remove('שד')
            if 'שדרות' in cleaned_string:
                cleaned_string.remove('שדרות')
            cleaned_string.sort(reverse=True)
            string_template = ' '.join(cleaned_string)
            result.loc[index, 'streets'] = string_template.strip()    
    return result

##### The next function help us to sort the addresses and compre between them.

In [12]:
def get_addresses_name(streets):
    streets_name = []
    for address in streets:
        parts = address.split()
        if parts[-1].isdigit():
            parts.pop()
        street_name = ' '.join(parts)
        streets_name.append(street_name)
    return streets_name

#### The next function creates a dataframe that includes all the parameters we collected:
1. The information about the apartment
2. The last deals about the apartment spreated by years from 2010 to 2023
3. If the apartment's buliding is in tma38 proccess it will get the value 1
4. If the apartment's buliding is in tma38 type 2 proccess it will get the value 2
5. If the apartment's buliding is in tma38 type 2 proccess it will get the value 3
6. If the apartment's buliding is in construction_evacuation proccess it will get the value 4

In [28]:
def add_columns(file_name, city, h_city):
    df = pd.read_csv(file_name, encoding='hebrew')
    df.drop(['Unnamed: 0', 'index'], axis=1, inplace=True)
    df.drop_duplicates(subset=['prices', 'streets', 'cities', 'floors', 'sizes'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    #add columns to the df:
    indexes = ['none' for _ in range(0, df.shape[0])]
    df['construction_evacuation'] = [0 for _ in range(0, df.shape[0])]
    for year in range(2010, 2024):
        df[f'{year}'] = indexes
    #sorts the addresses values in the df:
    df = remove_characters(df)
    df.sort_values(by='streets', inplace=True)
    df.reset_index(drop=True, inplace=True)
    streets = df["streets"].values
    #fill the tma38 column:
    tma_dict = tma_df(h_city) #dynamic crawling
    clean_tma_dict = remove_characters(tma_dict)
    for index, address in enumerate(streets):
        for key, value in clean_tma_dict.items():
            if address == key:
                df.loc[index, "construction_evacuation"] = value
    #fill the construction_evacuation column:
    ce_list = get_Construction_evacuation_complexes(h_city) #dynamic crawling
    if len(ce_list) > 0:
        clean_ce = remove_characters(ce_list) 
        ce_set = set(clean_ce)
        for index, address in enumerate(streets):
            if address in ce_set:
                df.loc[index, "construction_evacuation"] = 4
            elif address > clean_ce[-1]:
                break
    #fill the years columns:
    deals_df = pd.read_csv(f"C:\\Users\\Tal\Desktop\\לימודים\מדעי נתונים\\last_deals_expirience\\fixed_{city}_3_rooms.csv", encoding="hebrew")
    deals_df.rename(columns={'street': 'streets'}, inplace=True)
    deals_df.drop(['Unnamed: 0'], axis=1, inplace=True)
    deals_df = remove_characters(deals_df)
    deals_df.sort_values(by='streets', inplace=True)
    deals_df.drop_duplicates(inplace=True)
    deals_df.reset_index(drop=True, inplace=True)
    deals_df.to_csv(f"C:\\Users\\Tal\Desktop\\לימודים\מדעי נתונים\\last_deals_expirience\\final_file_{city}_3_rooms.csv", encoding="hebrew")
    streets_name = []
    count= [1 for _ in range(14)]
    streets_name = get_addresses_name(streets)
    deals_streets_name = get_addresses_name(deals_df['streets'].values)
    j = 0
    i = 0
    k = 0
    r = 0 #the r index helps improve the time running
    #checkks if theres any deals in the df for the apartment and clac the average:
    while i < df.shape[0] and j < deals_df.shape[0]:
        if k >= deals_df.shape[0]:
            break
        elif streets_name[i] == deals_streets_name[j]:
            k = r
            while k < deals_df.shape[0]:
                #checks if theres deals of the specific address and number address:
                if streets[i] == deals_df.loc[k, 'streets'] and streets[i] != streets_name[i]:
                    for year in range(2010, 2024):
                        df.loc[i, f'{year}'] = deals_df.loc[k, f'{year}']
                    count= [1 for _ in range(14)]
                    i += 1
                    break 
                #if there isnt any deals for the specific building then calculate all the street avrege deals:
                elif streets_name[i] == deals_streets_name[k]:   
                    for year in range(2010, 2024):
                        s1 = str(df.loc[i, f'{year}'])
                        s2 = str(deals_df.loc[k, f'{year}'])
                        if s1 != 'none' and s2 != 'none' and s1 != 'nan' and s2 != 'nan':
                                df.loc[i, f'{year}'] = int(s1.replace("," , "")) + int(s2.replace("," , ""))
                                count[year-2010] += 1
                        elif deals_df.loc[k, f'{year}'] != 'none' and  df.loc[i, f'{year}'] == 'none':
                            df.loc[i, f'{year}'] = deals_df.loc[k, f'{year}']
                    k += 1
                else:
                    for index, c in enumerate(count):
                        if c > 1:
                            df.loc[i, f'{index+2010}'] = int(df.loc[i, f'{index+2010}'] / c)                  
                    count= [1 for _ in range(14)]
                    i += 1
                    break        
        elif  streets_name[i] > deals_streets_name[j]:
            if 0 < k < deals_df.shape[0]:
                j = k
                k = 0
            else:
                j += 1
                r = j
        else:
            i += 1
    return df

##### The next function calls to add_columns and creates a final data frame for all the cities at the cities list:

In [ ]:
def creating_final_files():
    #values for crawling and keys for creating files:
    cities = {'jerusalem':'ירושלים', 'tel-aviv':'תל אביב', 'haifa':'חיפה', 'rishon-lezion':'ראשון לציון', 'petah-tikva':'פתח תקווה',
               'netanya':'נתניה', 'holon':'חולון', 'beer-sheva':'באר שבע', 'ashdod':'אשדוד','bnei-brakh':'בני ברק'}
                 
    for key, value in cities.items():
        df = add_columns(f"C:\\Users\\Tal\\Desktop\\לימודים\\מדעי נתונים\\apartments_info\\{key}_df_file_3_rooms.csv", key, value) 
        df.to_csv(f"C:\\Users\\Tal\\Desktop\\לימודים\\מדעי נתונים\\final_df\\{key}_df_file_3_rooms.csv", encoding="hebrew") 
creating_final_files()

## **Data handling**
The next functions taking care the data in each data frame before the maching learning


#### The next function fills the none values of the years columns at the data frame with the price increase percentage per year.

In [ ]:
def calc_Price_increase_percentage_per_year():
    df = pd.read_csv("C:\\Users\\Tal\\Desktop\\לימודים\\מדעי נתונים\\final_df\\ashdod_df_file_3_rooms.csv", encoding="hebrew")
    averages_list = []
    
    for year in range(2010, 2023):
        diff_list = []
        
        for row in range(df.shape[0]):
            if df.loc[row, f"{year}"] != 'none' and df.loc[row, f"{year+1}"] != 'none' and not pd.isna(df.loc[row, f"{year}"]) and not pd.isna(df.loc[row, f"{year+1}"]): 
                s1 = str(df.loc[row, f"{year}"])
                s2 = str(df.loc[row, f"{year+1}"])
                
                if (float(s2.replace(",", "")) - float(s1.replace(",", ""))) != 0:
                    diff_list.append(float(s1.replace(",", "")) / (float(s2.replace(",", "")) - float(s1.replace(",", ""))) / 100)
        
        if len(diff_list) > 0:
            averages_list.append(stat.mean(diff_list))
    
    for year in range(2010, 2023):
        for row in range(df.shape[0]):
            if df.loc[row, f"{year}"] != 'none' and (df.loc[row, f"{year+1}"] == 'none' or pd.isna(df.loc[row, f"{year+1}"])) and not pd.isna(df.loc[row, f"{year}"]): 
                s1 = str(df.loc[row, f"{year}"])
                try:
                    df.loc[row, f"{year+1}"] = float(s1.replace(",", "")) * (1 + averages_list[year-2010])
                except IndexError:
                    pass
    
    for year in range(2022, 2010, -1):
        for row in range(df.shape[0]):
            if df.loc[row, f"{year}"] != 'none' and (df.loc[row, f"{year-1}"] == 'none' or pd.isna(df.loc[row, f"{year-1}"])) and not pd.isna(df.loc[row, f"{year}"]): 
                s1 = str(df.loc[row, f"{year}"])
                try:
                    df.loc[row, f"{year-1}"] = float(s1.replace(",", "")) / (1 + averages_list[year-2010])
                except IndexError:
                    pass
    
    df.to_csv("C:\\Users\\Tal\\Desktop\\לימודים\\מדעי נתונים\\Data_handling\\dh_ashdod_df_file_3_rooms.csv", encoding="hebrew")

calc_Price_increase_percentage_per_year()

#### The next function changes all the variabels at the data frame into numeric variables:

In [ ]:
def Conversion_to_numeric_variables():
    df = pd.read_csv("C:\\Users\\yehud\\Downloads\\מבוא מדעי הנתונים\\project\\dh_ashdod_df_file_3_rooms.csv", encoding="hebrew")
    for row in range(0, df.shape[0]):
        if df.loc[row, 'terraces'] == 'none' or df.loc[row, 'terraces'] == '' or pd.isna(df.loc[row, 'terraces']):
            df.loc[row, 'terraces'] = 0
        else:
            df.loc[row, 'terraces'] = int(df.loc[row, 'terraces'][-1])
       
        if df.loc[row, 'floors_in_building'] == 'none' or df.loc[row, 'floors_in_building'] == '' or pd.isna(df.loc[row, 'floors_in_building']):
            df.loc[row, 'floors_in_building'] = 0
        else:
            df.loc[row, 'floors_in_building'] = int(df.loc[row, 'floors_in_building'][-1])
        
        
        if df.loc[row, 'parkings'] == 'none' or df.loc[row, 'parkings'] == '' or pd.isna(df.loc[row, 'parkings']) or 'ללא' in df.loc[row, 'parkings']:
            df.loc[row, 'parkings'] = 0  
        else:
            df.loc[row, 'parkings'] = int(df.loc[row, 'parkings'][-1])
            
        if df.loc[row, 'floors'] == 'none' or df.loc[row, 'floors'] == '' or pd.isna(df.loc[row, 'floors']) or 'קרקע' in df.loc[row, 'floors']:
            df.loc[row, 'floors'] = 0  
    
  
        if df.loc[row, 'status'] == 'none' or df.loc[row, 'status'] == '' or pd.isna(df.loc[row, 'status']):
            if df.loc[row, 'renovated'] == 1:
                df.loc[row, 'status'] = 3
            else:
                df.loc[row, 'status'] = 0
        elif 'דרוש שיפוץ' in df.loc[row, 'status']:
            df.loc[row, 'status'] = 1
        elif 'שמור' in df.loc[row, 'status']:
            df.loc[row, 'status'] = 2
        elif 'משופץ' in df.loc[row, 'status']:
            df.loc[row, 'status'] = 3
        elif 'חדש' in df.loc[row, 'status']:
            df.loc[row, 'status'] = 4
       
        
        
        if df.loc[row, 'time_enter'] == 'none' or df.loc[row, 'time_enter'] == '' or pd.isna(df.loc[row, 'time_enter']) or 'לא צוין' in  df.loc[row, 'time_enter'] :
            df.loc[row, 'time_enter']=0
        elif 'כניסה גמישה' in df.loc[row, 'time_enter'] or '202' in  df.loc[row, 'time_enter']:
            df.loc[row, 'time_enter']=1
        elif 'כניסה מיידית' in df.loc[row, 'time_enter']:  
             df.loc[row, 'time_enter']=2
    
    df.to_csv("C:\\Users\\yehud\\Downloads\\מבוא מדעי הנתונים\\project\\dh_ashdod_df_file_20_rooms.csv", encoding="hebrew")


Conversion_to_numeric_variables()

#### The next function delete all the rows that ave no last deals or doesnt have properties about the apartment

In [68]:
def delete_empty_rows():
    df = pd.read_csv("C:\\Users\\yehud\\Downloads\\מבוא מדעי הנתונים\\project\\dh_ashdod_df_file_20_rooms.csv", encoding="hebrew")
    for row in range(0,df.shape[0]):
        if df.loc[row, '2010']== 'none':
            df.drop(row , inplace=True)
    df.dropna(axis=0, inplace=True)
    df.reset_index(drop= True)
    df.to_csv("C:\\Users\\yehud\\Downloads\\מבוא מדעי הנתונים\\project\\dh_ashdod_df_file_rooms.csv", encoding="hebrew")
    
delete_empty_rows()    

1


In [ ]:
df = pd.read_csv("C:\\Users\\yehud\\Downloads\\מבוא מדעי הנתונים\\project\\dh_ashdod_df_file_rooms.csv", encoding="hebrew") 

df.drop(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'], axis=1 , inplace =True)
for row in range(0,df.shape[0]):
    if 'צוין' in df.loc[row , 'prices'] or pd.isna(df.loc[row , 'prices']):
        df.loc[row , 'prices']=0
    else:
        if '₪' in df.loc[row , 'prices'] or "¤" in df.loc[row , 'prices']  :
            subset=df.loc[row , 'prices'][0:-1]
            s1 = subset.replace(",","")
            print(s1)
            df.loc[row , 'prices'] =int(s1)   
        else:
            s1 =str(df.loc[row , 'prices'])
            s1 = s1.replace(",","").strip()
            df.loc[row , 'prices'] =int(s1)   
sns.boxplot(df['2010'])

## **Data handling**
The next functions taking care the data in each data frame before the maching learning


#### The next function fills the none values of the years columns at the data frame with the price increase percentage per year.

In [ ]:
def calc_Price_increase_percentage_per_year():
    df = pd.read_csv("C:\\Users\\yehud\\Desktop\\project 23\\dh_ashdod_df_file_3_rooms.csv", encoding="hebrew")
    averages_list = []
    
    for year in range(2010, 2023):
        diff_list = []
        
        for row in range(df.shape[0]):
            if df.loc[row, f"{year}"] != 'none' and df.loc[row, f"{year+1}"] != 'none' and not pd.isna(df.loc[row, f"{year}"]) and not pd.isna(df.loc[row, f"{year+1}"]): 
                s1 = str(df.loc[row, f"{year}"])
                s2 = str(df.loc[row, f"{year+1}"])
                
                if (float(s2.replace(",", "")) - float(s1.replace(",", ""))) != 0:
                    diff_list.append(float(s1.replace(",", "")) / (float(s2.replace(",", "")) - float(s1.replace(",", ""))) / 100)
        
        if len(diff_list) > 0:
            averages_list.append(stat.mean(diff_list))
    
    for year in range(2010, 2023):
        for row in range(df.shape[0]):
            if df.loc[row, f"{year}"] != 'none' and (df.loc[row, f"{year+1}"] == 'none' or pd.isna(df.loc[row, f"{year+1}"])) and not pd.isna(df.loc[row, f"{year}"]): 
                s1 = str(df.loc[row, f"{year}"])
                try:
                    df.loc[row, f"{year+1}"] = int(float(s1.replace(",", "")) * (1 + averages_list[year-2010]))
                except IndexError:
                    pass
    
    for year in range(2022, 2010, -1):
        for row in range(df.shape[0]):
            if df.loc[row, f"{year}"] != 'none' and (df.loc[row, f"{year-1}"] == 'none' or pd.isna(df.loc[row, f"{year-1}"])) and not pd.isna(df.loc[row, f"{year}"]): 
                s1 = str(df.loc[row, f"{year}"])
                try:
                    df.loc[row, f"{year-1}"] = int(float(s1.replace(",", "")) / (1 + averages_list[year-2010]))
                except IndexError:
                    pass
    
    df.to_csv("C:\\Users\\yehud\\Desktop\\project 23\\dh_ashdod_df_file_3_rooms.csv", encoding="hebrew")

calc_Price_increase_percentage_per_year()

In [ ]:
def Conversion_to_numeric_variables(city , hcity):
    df = pd.read_csv(f"C:\\Users\\yehud\\Desktop\\project 23\\dh_{city}_df_file_3_rooms.csv", encoding="hebrew")
    index_neighborhood=1
    dict_city={}
    for row in range(0, df.shape[0]):
        if df.loc[row, 'terraces'] == 'none' or df.loc[row, 'terraces'] == '' or pd.isna(df.loc[row, 'terraces']):
            df.loc[row, 'terraces'] = 0
        else:
            df.loc[row, 'terraces'] = int(df.loc[row, 'terraces'][-1])
       
        if df.loc[row, 'floors_in_building'] == 'none' or df.loc[row, 'floors_in_building'] == '' or pd.isna(df.loc[row, 'floors_in_building']):
            df.loc[row, 'floors_in_building'] = 0
        else:
            df.loc[row, 'floors_in_building'] = int(df.loc[row, 'floors_in_building'][-1])
        
        
        if df.loc[row, 'parkings'] == 'none' or df.loc[row, 'parkings'] == '' or pd.isna(df.loc[row, 'parkings']) or 'ללא' in df.loc[row, 'parkings']:
            df.loc[row, 'parkings'] = 0  
        else:
            df.loc[row, 'parkings'] = int(df.loc[row, 'parkings'][-1])
            
        if df.loc[row, 'floors'] == 'none' or df.loc[row, 'floors'] == '' or pd.isna(df.loc[row, 'floors']) or 'קרקע' in df.loc[row, 'floors']:
            df.loc[row, 'floors'] = 0  
    
  
        if df.loc[row, 'status'] == 'none' or df.loc[row, 'status'] == '' or pd.isna(df.loc[row, 'status']):
            if df.loc[row, 'renovated'] == 1:
                df.loc[row, 'status'] = 3
            else:
                df.loc[row, 'status'] = 0
        elif 'דרוש שיפוץ' in df.loc[row, 'status']:
            df.loc[row, 'status'] = 1
        elif 'שמור' in df.loc[row, 'status']:
            df.loc[row, 'status'] = 2
        elif 'משופץ' in df.loc[row, 'status']:
            df.loc[row, 'status'] = 3
        elif 'חדש' in df.loc[row, 'status']:
            df.loc[row, 'status'] = 4
         

        if df.loc[row, 'cities'] == 'none' or df.loc[row, 'cities'] == '' or pd.isna(df.loc[row, 'cities']):
            df.loc[row, 'cities']=df.loc[row, 'cities'] = 0
        else:
            if 'דירה' in df.loc[row, 'cities']: 
                df.loc[row, 'cities']=df.loc[row, 'cities'].replace('דירה', "").replace(",","")

            if hcity in df.loc[row, 'cities']:
                df.loc[row, 'cities']=df.loc[row, 'cities'].replace(hcity, "").replace(",","")
        
        if df.loc[row, 'cities'] not in dict_city:
            dict_city[df.loc[row, 'cities']] = index_neighborhood
            df.loc[row, 'cities']=dict_city[df.loc[row, 'cities']]
            index_neighborhood+=1
        else:
            df.loc[row, 'cities']=dict_city[df.loc[row, 'cities']]
        
        
        if df.loc[row, 'time_enter'] == 'none' or df.loc[row, 'time_enter'] == '' or pd.isna(df.loc[row, 'time_enter']) or 'לא צוין' in  df.loc[row, 'time_enter'] :
            df.loc[row, 'time_enter']=0
        elif 'כניסה גמישה' in df.loc[row, 'time_enter'] or '202' in  df.loc[row, 'time_enter']:
            df.loc[row, 'time_enter']=1
        elif 'כניסה מיידית' in df.loc[row, 'time_enter']:  
             df.loc[row, 'time_enter']=2
    
    df.to_csv("C:\\Users\\yehud\\Downloads\\מבוא מדעי הנתונים\\project\\dh_ashdod_df_file_20_rooms.csv", encoding="hebrew")


Conversion_to_numeric_variables('ashdod', 'אשדוד')

In [ ]:
def delete_empty_rows():
    df = pd.read_csv("C:\\Users\\yehud\\Downloads\\מבוא מדעי הנתונים\\project\\dh_ashdod_df_file_20_rooms.csv", encoding="hebrew")
    for row in range(0,df.shape[0]):
        if df.loc[row, '2010']== 'none':
            df.drop(row , inplace=True)
    df.dropna(axis=0, inplace=True)
    df.reset_index(drop= True)
    df.to_csv("C:\\Users\\yehud\\Downloads\\מבוא מדעי הנתונים\\project\\dh_ashdod_df_file_rooms.csv", encoding="hebrew")
    
delete_empty_rows()    


In [ ]:
df = pd.read_csv("C:\\Users\\yehud\\Downloads\\מבוא מדעי הנתונים\\project\\dh_ashdod_df_file_rooms.csv", encoding="hebrew")
df.drop(["Unnamed: 0"], axis=1 , inplace =True)
df.to_csv("C:\\Users\\yehud\\Downloads\\מבוא מדעי הנתונים\\project\\dh_ashdod_df_file_rooms.csv", encoding="hebrew")

In [ ]:
df = pd.read_csv("C:\\Users\\yehud\\Downloads\\מבוא מדעי הנתונים\\project\\dh_ashdod_df_file_rooms.csv", encoding="hebrew") 

box_l= ['prices','sizes' , '2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020' ]
for row in range(0,df.shape[0]):
    if  pd.isna(df.loc[row , 'prices']) :
        df.loc[row , 'prices']=0
        print(type(df.loc[row , 'prices']))
    elif  type(df.loc[row , 'prices']) ==str and 'צוין' in df.loc[row , 'prices'] :
        df.loc[row , 'prices']=0
       
            
        
        print(type(df.loc[row , 'prices']))
        
    else:
        if '₪' in df.loc[row , 'prices'] or "¤" in df.loc[row , 'prices']  :
            subset=df.loc[row , 'prices'][0:-1]
            s1 = subset.replace(",","")
            df.loc[row , 'prices'] =int(s1)  
           # print(type(df.loc[row , 'prices']))
        else:
            s1 =str(df.loc[row , 'prices'])
            s1 = s1.replace(",","").strip()
            df.loc[row , 'prices'] =int(s1)
            print(type(df.loc[row , 'prices']))
            

            
            
   
           
    for col in df.columns:
        if col == 'streets' or col == '2021'or col == '2022' or col == '2023' :
            continue
        df.loc[row , col]=int(df.loc[row , col])
for row in range(0,df.shape[0]):
    avg_l=[]
    for index in range(0,df.shape[0]):
        if df.loc[row, 'cities']== df.loc[index, 'cities'] and df.loc[row, 'prices']== 0 and df.loc[index, 'prices']!= 0:##############
            avg_l.append(df.loc[index, 'prices'])  
    if avg_l:
        df.loc[row, 'prices'] = statistics.mean(avg_l)       
            
    
df['prices'] = df['prices'].astype(float)

columns = ['prices', 'sizes', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
           'cities', 'floors', 'terraces', 'floors_in_building', 'parkings', 'aircondition', 'bars',
           'elevators', 'water_headers', 'accessibility', 'shelters', 'renovated', 'storages', 'tadiran',
           'furnitures', 'status', 'time_enter', 'sea_view', 'sukka', 'construction_evacuation']
col_numeric=['prices', 'sizes', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
             'cities', 'floors' ]
          

df_subset = df[col_numeric].astype(float).copy()# Select the specified columns from your DataFrame
print(df_subset.corr())
pd.plotting.scatter_matrix(df_subset, figsize=(50, 50), diagonal='kde')
plt.show()
df.to_csv("C:\\Users\\yehud\\Downloads\\מבוא מדעי הנתונים\\project\\dh_ashdod_df_file_rooms100.csv", encoding="hebrew")

In [ ]:
columns_categorial = ['prices', 'terraces', 'floors_in_building', 'parkings', 'aircondition', 'bars',
           'elevators', 'water_headers', 'accessibility', 'shelters', 'renovated', 'storages', 'tadiran',
           'furnitures', 'status', 'time_enter', 'sea_view', 'sukka', 'construction_evacuation']
df_subset = df[columns_categorial].astype(float).copy()# Select the specified columns from your DataFrame
print(df_subset.dtypes)
print(df_subset.corr())
pd.plotting.scatter_matrix(df_subset, figsize=(50, 50), diagonal='kde')
plt.show()

In [ ]:
columns_categorial = [ 'terraces', 'floors_in_building', 'parkings',
           'elevators', 'accessibility', 'shelters', 'storages', 'sea_view']
for col in columns_categorial:
    sns.violinplot(x=df[col], y=df['prices'])
    plt.xlabel(col)
    plt.ylabel('Prices')
    plt.show()

In [ ]:
def replace_outliers_with_median(df, column_name):
    median = df[column_name].mean()
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    df[column_name] = df[column_name].apply(lambda x: median if x < lower_bound or x > upper_bound else x)
    

In [ ]:
def split_to_train_and_test(X, y, test_ratio, rand_state):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=rand_state)
    return X_train,X_test,y_train,y_test

In [ ]:
def load_dataset(file_name, target_column):
    df= pd.read_csv(file_name)
    data_list = ['prices', 'cities', 'floors', 'sizes', 'terraces', 'floors_in_building', 'parkings', 'aircondition', 'bars', 'elevators', 'water_headers', 'accessibility', 'shelters', 'renovated', 'storages', 'tadiran', 'furnitures', 'status', 'time_enter', 'sea_view', 'sukka', 'construction_evacuation', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
    df_selected = df[data_list]
    new_series= pd.Series(df_selected[target_column])
    df_selected=df_selected.drop(target_column,axis =1)
    return df,new_series